# Day type identification of Algerian electricity load in Annaba city

## Introduction

In the domain of energy management and consumption, understanding the patterns and characteristics of electricity load is crucial for efficient resource utilization and future planning. This data mining project focuses on analyzing the Day Type Identification of electricity load in Annaba City, Algeria.

Annaba, one of the rich cities in industrial and commercial landscapes in Algeria, it experiences diverse electricity consumption patterns influenced by factors such as industrial activities, residential needs, and commercial enterprises. Analyzing and categorizing these patterns into distinct day types can provide valuable insights for optimizing energy distribution, predicting peak loads, and enhancing overall energy sustainability.
### Objective

One of the challenges faced in energy management is the dynamic nature of electricity consumption, influenced by various factors. Unpredictable times of high demand, inefficient resource allocation, and the lack of a comprehensive understanding of consumption patterns, all this can lead to bad energy distribution over time. Our objective is to address these challenges by leveraging data mining techniques to identify distinct day types based on electricity load. By categorizing days into specific types such as weekdays, seasons, holidays, or special events, we aim to provide a nuanced understanding of consumption dynamics. This, in turn, can contribute to the resolution of problems related to inefficient resource allocation, unexpected demand fluctuations, and the lack of a tailored approach to energy distribution.

### Dataset

We'll be working with the 'pma.csv' dataset, it contains in every hour how much electricity it is consumed and the temperature at that time, from the start of the year 2016 to the end of 2017.

### Project steps:

- Feature Engineering.
- Data cleaning
- Exploratory data analysis
- Data preprocessing
- Modeling: Regression
- Modeling: clustering

More detail about the steps and the code of it are below.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['axes.grid']=True

In [ ]:
ts_data = pd.read_excel("pma.xlsx")
ts_data = ts_data.reset_index().drop("index", axis=1)
ts_data["time"] = pd.to_datetime(ts_data["time"])
ts_data = ts_data.set_index("time")
ts_data.head()

### Feature Engineering

Here we will devide the column of time into three parts: date and hour.

Also, we will add more columns to our dataset, which are
- season
- is_special_day
- dayofmonth
- dayofyear
- year
- month
- quarter
- dayofweek

In [ ]:
seasons = [1, 2, 3, 4]

def map_season(month, day):
    if (month == 12 and day >= 21) or (month == 1) or (month == 2) or (month == 3 and day < 21):
        return seasons[0]  # Winter
    elif (month == 3 and day >= 21) or (month == 4) or (month == 5) or (month == 6 and day < 21):
        return seasons[1]  # Spring
    elif (month == 6 and day >= 21) or (month == 7) or (month == 8) or (month == 9 and day < 21):
        return seasons[2]  # Summer
    else:
        return seasons[3]  # Fall


def is_special_day(date):
    special_days = ['2016-07-06', '2017-06-25', '2016-09-12', '2017-09-01']
    special_days = pd.to_datetime(special_days).date

    return int(date in special_days)


def create_features(df):
    """
    Create time series features based on the time series index.
    """
    df = df.copy()
    df["date"] = df.index.date
    df['hour'] = df.index.hour
    df['dayofweek'] = ((df.index.dayofweek + 8) % 7) + 1
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df["is_special_day"] = df.apply(lambda row: is_special_day(row.date), axis=1)
    df['season'] = df.apply(lambda row: map_season(row.month, row.dayofmonth), axis=1)

    return df

df = create_features(ts_data)

df.head()

### Data cleaning

#### Checking missing data and duplicated

In [ ]:
total_missing_values = df.isna().sum().sum()
total_duplicates = df.duplicated().sum()

print(f'total of missing values: {total_missing_values}')
print(f'total of duplicated values: {total_duplicates}')

### EDA

#### General overview of the power demand over time

In [ ]:
plt.plot(df.index, df["pma"])

### Trends

#### PMA over all time

Here, we will see the development of the power demand over time in 2016 and 2017 by calculating the mean power demand on each month

In [ ]:
year_month_grouped_pma = df.groupby(['year', 'month']).pma.agg(years_monthly_pma='mean')

title_style = {'family':'serif','color':'darkblue','size':18, 'weight':'bold'}
labels_style = {'family':'serif','color':'black','size':15}
def line_plot(data, x, y, title='', xlabel='', ylabel='', rotate_x=0):
    plt.figure(figsize=[12, 6])
    plt.title(title, fontdict=title_style)
    plt.ylabel(ylabel, fontdict=labels_style)
    plt.xlabel(xlabel, fontdict=labels_style)
    plt.xticks(rotation=rotate_x)
    sns.lineplot(data=data, x=x, y=y)
    plt.show()

# adding a column for the combination year-month
year_month_grouped_pma['year_month'] = year_month_grouped_pma.index.map(lambda x: f'{x[0]}' + '-' + f'{x[1]}')
# year_month_grouped_pma

# plotting the data
line_plot(year_month_grouped_pma, 'year_month', 'years_monthly_pma', 'monthly pma between 2016 and 2017', 'year-month', 'pma', 90)

#### PMA over months

As we can notice earlier, the maximum power demand increases mostly during summer from May till October.  
To see this better, let us take the average pma between the two years for each month:

In [ ]:
monthly_grouped_pma = year_month_grouped_pma.groupby('month').years_monthly_pma.agg(average_monthly_pma='mean')
# monthly_grouped_pma
line_plot(monthly_grouped_pma, 'month', 'average_monthly_pma', 'Average Monthly pma', 'months', 'pma')

In [ ]:
sns.boxplot(data=df, x='month', y='pma')

#### PMA over days

Now, let explore pma in days, as we did in months before, we calculate the mean power demand of each hour in the day

In [ ]:
hourly_grouped_pma = df.groupby('hour').pma.agg(hourly_pma='mean')
#hourly_grouped_pma
line_plot(hourly_grouped_pma, 'hour', 'hourly_pma', 'Average Hourly PMA During One Day', 'Hour', 'pma')

In [ ]:
sns.boxplot(data=df, x='hour', y='pma')

we can notice that the consumption of electricity starts decreasing in the morning till it achives its minimum. then it gets higher and higher till it reaches its maximum at night.

However, this analysis is considering all the days in both two years, let us split the days according to the season it belongs to and try to compare.

In [ ]:
seasoned_hourly_grouped_pma = df.groupby(['season', 'hour']).pma.agg(seasoned_hourly_pma='mean')
fall_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Fall']
winter_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Winter']
spring_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Spring']
summer_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')=='Summer']

plt.figure(figsize=[14, 6])
plt.title('Average Hourly PMA During Each Season')
plt.ylabel('seasoned pma')
plt.xlabel('hour')

season_num_to_name = {
    1: "Winter",
    2: "Spring",
    3: "Summer",
    4: "Fall"
}

for season in seasons:
    seasoned_hourly = seasoned_hourly_grouped_pma[seasoned_hourly_grouped_pma.index.get_level_values('season')==season]
    #line_plot(seasoned_hourly, 'hour', 'seasoned_hourly_pma', f'Average Hourly Demand in {season}', 'Hour', 'pma')
    sns.lineplot(data=seasoned_hourly, x='hour', y='seasoned_hourly_pma', label=season_num_to_name[season])

plt.show()

From this graph, we notice that the consumption in summer is the highest, followed by fall and winter, then spring

Another thing we can see here is unlike other seasons, in the summer, the consumption gets higher in the middle of the day (from around 1pm to 4pm)
this can be due to the use of air conditioners, because of the high temperature at that time of the day

In [ ]:
sns.boxplot(data=df, x='season', y='pma')

As we saw earlier, the consumption in the summer is the highest, followed by winter and fall, then spring

#### PMA in Week Days

We'll now discover which day in the week has the highest consumption of energy

In [ ]:
# We plot the average usage of each day of the week
dayName_grouped_pma = df.groupby('dayofweek').pma.agg(average_pma='mean')

# We sort the days of the week
dayName_grouped_pma = dayName_grouped_pma.reindex(range(8))

# We plot the data
line_plot(dayName_grouped_pma, 'dayofweek', 'average_pma', 'Average pma During Each Day of the Week', 'Day of the Week', 'pma')


We notice that the consumption is lower in the weekends, this is probably due to reduced industrial and commercial activities, especially in Friday (day 6); we find almost all supermarkets, industries, schools, closed at that time. 

#### PMA in special days (Eid Days)

Let us discover how was the usage of people in "Eids", we might find some high usage due to family gathering and social and cultural activities



In [ ]:
# We first define the special days (Eids in 2016 and 2017)
special_days_df = df.loc[df["is_special_day"] == 1]
special_days_df

Let's compare the usage in Eid days to the overall usage

In [ ]:
# We plot a comparison between the average hourly pma in the special days and the average hourly pma in the normal days
normal_days_df = df[df["is_special_day"] == 0]
normal_days_hourly_grouped_pma = normal_days_df.groupby('hour').pma.agg(normal_hourly_pma='mean')
special_days_hourly_grouped_pma = special_days_df.groupby('hour').pma.agg(special_hourly_pma='mean')
# we plot the comparison
plt.figure(figsize=[12, 6])
plt.title('Average Hourly PMA During Special Days and Normal Days', fontdict=title_style)
plt.ylabel('pma', fontdict=labels_style)
plt.xlabel('hour', fontdict=labels_style)
sns.lineplot(data=normal_days_hourly_grouped_pma, x='hour', y='normal_hourly_pma', label='normal days')
sns.lineplot(data=special_days_hourly_grouped_pma, x='hour', y='special_hourly_pma', label='special days')
plt.show()

sns.boxplot(data=df, x='is_special_day', y='pma')

It's remarkable that the usage in Eids is higher than in the normal days due to the increased social Activities and family gathering .
the consumption gets higher in the middle of the day (from around 1pm to 4pm) because most of Eids were in the summer in both years 2016 and 2017

### PMA in Ramadan

In [ ]:
# A dataframe containing the PMA of Ramadan days of 2016 and 2017 knowing that Ramadan days are from 6th of June to 5th of July in 2016 and from 26th of May to 24th of June in 2017
ramadan_df_2016 = df.loc[(df.index.date >= pd.to_datetime('2016-06-06').date()) & (df.index.date <= pd.to_datetime('2016-07-05').date())]
ramadan_df_2017 = df.loc[(df.index.date >= pd.to_datetime('2017-05-26').date()) & (df.index.date <= pd.to_datetime('2017-06-24').date())]
ramadan_df = pd.concat([ramadan_df_2016, ramadan_df_2017])
ramadan_df



In [ ]:
# Plotting the average hourly pma during Ramadan days and the average hourly pma during normal days
normal_days_df = df.loc[(df.index.date < pd.to_datetime('2016-06-06').date()) | (df.index.date > pd.to_datetime('2016-07-05').date())]
normal_days_df = normal_days_df.loc[(normal_days_df.index.date < pd.to_datetime('2017-05-26').date()) | (normal_days_df.index.date > pd.to_datetime('2017-06-24').date())]
normal_days_hourly_grouped_pma = normal_days_df.groupby('hour').pma.agg(normal_hourly_pma='mean')
ramadan_days_hourly_grouped_pma = ramadan_df.groupby('hour').pma.agg(ramadan_hourly_pma='mean')
# we plot the comparison
plt.figure(figsize=[12, 6])
plt.title('Average Hourly PMA During Ramadan Days and Normal Days', fontdict=title_style)
plt.ylabel('pma', fontdict=labels_style)
plt.xlabel('hour', fontdict=labels_style)
plt.xticks(rotation=0)
plt.yticks(rotation=0)
sns.lineplot(data=normal_days_hourly_grouped_pma, x='hour', y='normal_hourly_pma', label='normal days')
sns.lineplot(data=ramadan_days_hourly_grouped_pma, x='hour', y='ramadan_hourly_pma', label='ramadan days')
plt.show()

We notice that the consumption gets higher then usual in ramadan at night (from 8pm to 3pm), and it gets really low in the early morning (from 4am to 10am), this is due to many people not sleeping at the start of the night to go to tarawih, practice night prayer, or simply just waiting for sahour time. 

#### Key notes for EDA step

* Power demand reaches its maximum value during summer. Which is logical, the high temperature at that time leads to the higher use of air conditioners.
* During the day, PMA reaches its max at around 8pm.
* During Winter, max demand is at its peak before 8pm, and after that time it starts decreasing. This could be due to many reasons. One of them is that people tend to sleep earlier at winter. Meanwhile during summer, it reaches its max after 8pm and we notice higher consumption as well between 1pm and 4pm.
* The demand increases in summer in the period from 1pm to 4pm, this can be due to the use of air conditioners, because of the high temperature at that time of the day.
* Demand is low in all seasons during early morning and medium during the middle of the day where everyone are doing their activities and daily tasks.
* Demand is low in weekends (especially in Fridays) due to reduced industrial and commercial activities.
* Demand is higher in Eids because of social and cultural activities and family gathering which cause a high consumption of electricity.
* The consumption at night is higher then usual in Ramadan due to the fact that people sleep late waiting for sahour, going to tarawih and practicing night pray.

### Data preprocessing

#### Grouping by days
- Our data has each hour in a row, so we need to take the median of each 24 hours to represent that day
- We choose the median to avoid outliers' effect.

In [ ]:
df_daily = df.groupby("date").agg({"pma": "median", "tmp": "median", "year": "min", "month": "min", "dayofmonth": "min", "dayofyear": "min", "is_special_day": "min", "season": "min", "dayofweek": "min", "quarter": "min"})

df_daily.index = pd.to_datetime(df_daily.index)

df_daily.head()

#### Cyclic attribute representation
Since our data is a time series, we have many attributes that are cyclic, for example: months, which starts with 0 and ends with 11. We all know that month 11 (December) and month 1 (January) are right next to each other, but with this representation 11 is way bigger than 1, so if we apply any clustering or regression algorithm it will think that those two values are very appart from each other, which is not the case in reality. This will lead to a fail to determine the real patterns in the data.

In [ ]:
def cyclical_transormation(df, columns: list, remove_original=False):
    df = df.copy()
    for col in columns:
        max_col_val = df[col].max()
        df[f"{col}_sin"] = np.sin((2*np.pi*df[col]) / max_col_val)
        df[f"{col}_cos"] = np.cos((2*np.pi*df[col]) / max_col_val)

    if remove_original:
        df = df.drop(columns=columns)

    return df

In [ ]:
cyclical_columns = ["year", "month", "dayofmonth", "dayofyear", "season", "dayofweek", "quarter"]
df_cyclic = cyclical_transormation(df_daily, columns=cyclical_columns, remove_original=True)
df_cyclic.head()

## Modeling

### PMA Regression to find most important features in order to justify the assumptions above

We create the function "train_test_split" in order to split our data into two sets, one for training and one for testing, the way we are doing this is by taking the variable "day_threshold" into account, which allows us to customize the interval between testing set entries. This parameter ensures that the division of consecutive days between the training and testing sets reflects the temporal dependencies present in our time-series data. For example: if the day_threshold equal to 4, the testing data will be the day number 4, 8, 12, 16, 20 ... and so on.

In [ ]:
def train_test_split(df, day_threshold=4):
    train = df.loc[(df.reset_index().index + 1) % day_threshold != 0]
    test = df.loc[(df.reset_index().index + 1) % day_threshold == 0]

    return train, test

We try to find the best hyper parameters set that leads to the best training results, we do that by considering all possible compinations

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

cols = df_cyclic.columns
TARGET = 'pma'
FEATURES = cols[cols != TARGET]

train, test = train_test_split(df_cyclic)

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

gsc = RandomizedSearchCV(
            estimator=xgb.XGBRegressor(),
            param_distributions={
                        "learning_rate": [0.005, 0.01, 0.02, 0.05, 0.1],
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "n_estimators": [2000, 3000, 5000, 10000],
                        "colsample_bytree":[ 0.3, 0.4],
                        },
            cv=3, scoring='neg_mean_squared_error', verbose=100, n_jobs=-1)

grid_result = gsc.fit(X_train, y_train)


In [ ]:
print('\n All results:')
print(grid_result.cv_results_)
print('\n Best estimator:')
print(grid_result.best_estimator_)
print('\n Best hyperparameters:')
print(grid_result.best_params_)

Now we initialize our model with the chosing set of hyper parameters and we train it

In [ ]:

reg = xgb.XGBRegressor(
    n_estimators=grid_result.best_params_["n_estimators"],
    max_depth=grid_result.best_params_["max_depth"],
    colsample_bytree=grid_result.best_params_["colsample_bytree"],
    min_child_weight=grid_result.best_params_["min_child_weight"],
    learning_rate=grid_result.best_params_["learning_rate"]
)


In [ ]:

reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

We order the features according to the most important ones, meaning the ones that have a bigger impact on the electricity consumption are first

In [ ]:
feature_importance_pd = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['importance'])
feature_importance_pd.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

##### Key Notes
- we see that Quarter, Season and temperature are one of the most important features that impacts pma values

In [ ]:
test['prediction'] = reg.predict(X_test)
df_pred = df_daily.merge(test[['prediction']], how='left', left_index=True, right_index=True)


In [ ]:

plt.plot(df_pred["pma"])
plt.plot(test["prediction"])
plt.legend(['Truth Data', 'Predictions'])
plt.title('Raw Data and Predictions')
plt.show()

In [ ]:
mse_score = mean_squared_error(test['pma'], test['prediction'])
rmse_score = np.sqrt(mse_score)

print(f'MSE Score on Test set: {mse_score:0.2f}')
print(f'RMSE Score on Test set: {rmse_score:0.2f}')

#### Key notes
- our regression model succeeds quiet well on predicting the values of the test data
- Using our model we can predict the PMA values of future records

### Clustering

#### K-Means clustering

We make clustering to our data acording to all columns except the pma one.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score

def pca_transform(df, explained_variance_threshold=0.95):
    pca = PCA(n_components=explained_variance_threshold)
    
    return pca.fit_transform(df)

def normalise(df):
    standard_scaler = StandardScaler()
    
    return standard_scaler.fit_transform(df)

In [ ]:
train, test = train_test_split(df_cyclic)

In [ ]:

# we remove "pma" since we probably don't know it in the future
# and so that our model doesn't depend on it in that case


train_no_pma = train.drop(columns=["pma"])
test_no_pma = test.drop(columns=["pma"])

We use silhouette score to find the best number of clusters

In [ ]:
def k_means_plot_silhouette_score(df, max_k=11):
    silhouette = []
    k_value = range(2, 11) # silhouette needs at least 2 clusters

    for k in k_value:
        kmeans = KMeans(n_clusters=k, n_init=10)
        kmeans.fit(df)
        cluster_labels = kmeans.labels_
        silhouette.append(silhouette_score(df, cluster_labels))

    plt.figure(figsize=[10, 6])
    plt.plot(k_value, silhouette, 'bx-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette')
    plt.title('Silhouette Metric To Find Optimal Number Of Clusters')
    plt.show()

    return k_value[np.array(silhouette).argmax(axis=0)]

def perform_kmeans_clustering(df, k):
    kmeans = KMeans(n_clusters=k, n_init=10)
    cluster = kmeans.fit(df)

    return cluster

In [ ]:
def pipeline(df, supposed_target="pma", k_override=None):
    no_target_df = df.drop(columns=[supposed_target])
    k_value = k_means_plot_silhouette_score(no_target_df) if k_override is None else k_override
    cluster = perform_kmeans_clustering(no_target_df, k=k_value)
    colormap = np.array(["r", "g", "b"])
    plt.scatter(df.index, df["pma"], c=colormap[cluster.labels_])
    print(f'SSE = {cluster.inertia_}')

In [ ]:
pipeline(df_cyclic, k_override=3)

We generated three clusters:

Red cluster (Low Usage):
    Happens in Fall and Spring.
    Makes sense because people don't use air conditioners or heaters much during these times.

Purple Group (Medium Usage):
    Mostly in winter.
    Winter nights are longer, so lights stay on more, the higher consumption here might also be due to the use of electric heaters.

Blue Group (High Usage):
    Mostly in summer.
    This is when people use air conditioners a lot.

# Conclusion
In our investigation into electricity consumption patterns in Annaba City, Algeria, we initiated a thorough exploration, beginning with feature engineering, data cleaning, and exploratory data analysis (EDA), then modeling with regression and clustering. Our primary goal was to determine specific day categories based on electricity load, aiming to uncover crucial insights for optimizing energy management and resource utilization.

**Key Findings:**
1. **Seasonal Consumption Patterns:**
   - We observed clear seasonal variations in electricity consumption, with higher demand during summer, attributable to the widespread use of air conditioners.
   - Winter months showed medium consumption, influenced by factors such as the use of electric heaters and longer nights resulting in increased lighting usage.

2. **Temporal Trends:**
   - The analysis of power demand over time revealed daily and monthly trends. Consumption tends to peak around 8 pm, and the summer months experience the highest demand.

3. **Special Events Impact:**
   - Consumption during special events like Eids and Ramadan showcased distinct patterns, with higher usage during these times, likely due to increased social activities, family gatherings, and altered daily routines.

4. **Weekday and Weekend Variation:**
   - Weekdays exhibited higher energy consumption compared to weekends, emphasizing the impact of industrial and commercial activities on electricity demand.
   - Fridays, in particular, showed a notable decrease in consumption, aligning with reduced business and school activities.

**Modeling (with regression):**
- We employed regression analysis to predict electricity consumption. Key features influencing consumption included the quarter, season, and temperature.
- The model created could be used to predict electricity consumption in the future.

**Modeling (with clustering):**
- K-Means clustering identified three clusters:
  - Red Cluster (Low Usage): Fall and Spring, associated with minimal use of air conditioners and heaters.
  - Purple Cluster (Medium Usage): Winter, marked by longer nights and increased lighting usage, possibly influenced by electric heaters.
  - Blue Cluster (High Usage): Summer, characterized by elevated demand due to extensive use of air conditioners.

**Implications:**
- Understanding these consumption patterns is crucial for optimizing energy distribution, predicting peak loads, and enhancing overall energy sustainability.
- Our findings can guide efficient resource allocation and aid in developing tailored approaches to energy distribution based on specific day types.

The insights derived from these analysis provide a valuable foundation for informed decision-making in energy management in Annaba or any similar Algerian City. One of the best dicisions we could make using these analysis is setting the price of electricity according to the expected amount of consumption at a specific time (if the consumption is high we make the price of the electricity higher), by doing this we will control the consumption during peak times and encourage people to use electricity in times when the consumption is low. This dynamic pricing strategy can lead to several benefits, such as preventing electricity cut-off because of high usage, electicity usage will be balanced throw out time, thus electricity destributing systems will have less problems in their work, and people that consider this dynamic pricing will save more money.


